In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-anatomy/sample_submission.csv
/kaggle/input/ai-anatomy/test_f.csv
/kaggle/input/ai-anatomy/train.csv


In [2]:
!pip install transformers


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
# from transformers import RobertaModel, RobertaTokenizer!
import logging
import re


In [5]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)


cuda


In [8]:
test = pd.read_csv('/kaggle/input/ai-anatomy/test_f.csv')
train = pd.read_csv('/kaggle/input/ai-anatomy/train.csv')


print(train.info())
print(train.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10479 entries, 0 to 10478
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        10479 non-null  int64 
 1   question  10479 non-null  object
 2   answer    10478 non-null  object
dtypes: int64(1), object(2)
memory usage: 245.7+ KB
None
     ID                                           question answer
0  8358  A 24-year-old man comes to the physician becau...     0%
1  5260  A 64-year-old male retired farmer presents to ...  0.002
2  7648  A healthy 29-year-old nulligravid woman comes ...  0.20%
3  1703  A 25-year-old man with a genetic disorder pres...     1%
4   945  A 14-month-old boy is brought in by his parent...  2.50%


In [ ]:

# we removed rows where 'answer' is purely numeric with no alphabetic values (e.g., 123 or 45.67)
train = train[~train['answer'].str.strip().str.match(r'^-?\d+(\.\d+)?$', na=False)]
# kept only answers containing at least 3 words
train = train[train['answer'].str.split().str.len() >= 3]
train.info()



<class 'pandas.core.frame.DataFrame'>
Index: 5643 entries, 83 to 10477
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        5643 non-null   int64 
 1   question  5643 non-null   object
 2   answer    5643 non-null   object
dtypes: int64(1), object(2)
memory usage: 176.3+ KB


In [ ]:
#checked for occurances of answers
train['answer'].value_counts()

answer
CT scan of the abdomen                         8
Iron deficiency anemia                         8
Autism spectrum disorder                       7
Patent ductus arteriosus                       6
Herpes simplex virus                           6
                                              ..
Erythrocyte transketolase activity             1
Erythocyte sedimentation rate of 65 mm/h\n"    1
Erythematous periorbital rash                  1
Eruption of mandibular incisors                1
Esmolol and glucagon                           1
Name: count, Length: 5237, dtype: int64

In [ ]:
print(train.duplicated(subset=['question', 'answer']).sum())

0


In [ ]:
train['answer_length'] = train['answer'].str.split().str.len()
print(train['answer_length'].describe())

count    5643.000000
mean       12.295056
std        32.456444
min         3.000000
25%         3.000000
50%         4.000000
75%         7.000000
max       322.000000
Name: answer_length, dtype: float64


In [ ]:
train.head(50)

,ID,question,answer,answer_length
83,2462,A cross-sectional oral health survey was desig...,...the difference between the observed and non...,10
84,5552,A 59-year-old man comes to the physician for a...,"""""""I apologize for the inconvenience, and if I...",23
85,6854,A 27-year-old woman with cystic fibrosis comes...,"""""""I understand that living with cystic fibros...",24
86,5282,A 27-year-old woman is brought to the physicia...,"""""""I understand that the sudden loss of your f...",32
87,6814,A 25-year-old man comes to the physician for s...,"""""""I understand that you are uncomfortable, bu...",26
88,4585,A 22-year-old man is brought to the emergency ...,"""""""I understand that you want to go home, but ...",21
89,3213,A 62-year-old woman is brought to the emergenc...,"""""""I was unable to obtain results from the ear...",21
90,7368,A 38-year-old woman comes to the physician for...,"""""""I would like to assess your symptoms causin...",17
91,3716,A 15-year-old girl comes to the physician for ...,"""""""I would like to discuss the various contrac...",12
92,2151,A previously healthy 73-year-old man comes to ...,"""""""I would like to know more about why you don...",16


In [15]:
train = train[~train['answer'].str.strip().str.endswith('?')]


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5641 entries, 83 to 10477
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             5641 non-null   int64 
 1   question       5641 non-null   object
 2   answer         5641 non-null   object
 3   answer_length  5641 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 220.4+ KB


In [ ]:
train['answer'].value_counts("")
keywords = ['AI', 'model', 'ai', 'large language model']
pattern = '|'.join([re.escape(word) for word in keywords])
count = train['answer'].str.contains(pattern, case=False, na=False).sum()
print(f"Number of answers containing any of {keywords}: {count}")
print(train.info())

Number of answers containing any of ['AI', 'model', 'ai', 'large language model']: 541
<class 'pandas.core.frame.DataFrame'>
Index: 5641 entries, 83 to 10477
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             5641 non-null   int64 
 1   question       5641 non-null   object
 2   answer         5641 non-null   object
 3   answer_length  5641 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 220.4+ KB
None


In [18]:
from datasets import Dataset

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(train)

In [ ]:
!pip install huggingface_hub

In [20]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf_token")


# Load token from environment variable
hf_token = secret_value_0

# Login to Hugging Face
login(token=hf_token)

In [21]:
hf_dataset.push_to_hub("defk0n1/aianatomy2")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/defk0n1/aianatomy2/commit/dbf7b92b61cf4957fbdbc5f32a1e8a4b3420167d', commit_message='Upload dataset', commit_description='', oid='dbf7b92b61cf4957fbdbc5f32a1e8a4b3420167d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/defk0n1/aianatomy2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='defk0n1/aianatomy2'), pr_revision=None, pr_num=None)

In [22]:
count = train['question'].str.strip().str.endswith('?').sum()
print(count)

5095
